In [1]:
import wget
import os
import pickle
import json

In [2]:
# # 定义文件路径

# 定义文件路径
url_txt_path = r"G:\Star\4_SpecTE-LAMOST_catalog\1_download_and_preprocessing_LAMOSTDR11\catalog_download\download_url\13-16.txt"     # URL文件
download_path = r"G:\Star\4_SpecTE-LAMOST_catalog\1_download_and_preprocessing_LAMOSTDR11\catalog_download\FITS\13-16"        # 目标下载位置
progress_file = r'G:\Star\4_SpecTE-LAMOST_catalog\1_download_and_preprocessing_LAMOSTDR11\catalog_download\progress\download_progress_13-16.json'    # 保存下载进度log文件（保存进度，用于恢复下载）

# url_txt_path = "./download_url/5-6.txt"
# download_path = "./Fits/5-6"
# progress_file = "./Fits/download_progress_5-6.json"

# 开始下载

In [3]:
# 保存下载进程
def save_progress(progress, progress_file):
    with open(progress_file, 'w') as f:
        json.dump(progress, f)

# 加载下载进程
def load_progress(progress_file):
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)
    return {'completed': [], 'err_log': {}, 'n': 0}

def download_spectra(url_txt_path, download_path, progress_file):
    with open(url_txt_path, 'r') as f:
        err_list = []
        ls = []

        # 读到链接列表ls
        f.readline()  # 跳过第一行，如果需要
        while True:
            line = f.readline()
            if line:
                ls.append(line.strip())  # 去掉换行符
            else:
                break

    # Number of all spectra
    print("Number of spectra to download:", len(ls))
    
    # 防止出问题
    progress = load_progress(progress_file)
    completed = set(progress['completed'])
    err_log = progress['err_log']
    n = progress['n']

    for j in range(0, len(ls)):
        if j in completed:
            continue
        try:
            wget.download(ls[j], download_path)
            completed.add(j)
            if j % 20 == 0:  # 每下载20个保存一次进度
                save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)
        except Exception as e:
            print(f"|| {j} err: {e}")
            err_list.append(j)
            continue

    err_log[n] = err_list
    n += 1
    save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)

    # 失败处重新下载
    while len(err_list) > 5:
        final_err_list = []
        print("\n重新下载失败光谱")
        print("Number of err spectra to download:", len(err_list))
        for j in err_list:
            try:
                wget.download(ls[j], download_path)
                completed.add(j)
                save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)
            except Exception as e:
                print(f"|| {j} err: {e}")
                final_err_list.append(j)
                continue

        err_log[n] = final_err_list
        err_list = final_err_list
        n += 1
        save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)

    return err_log

In [ ]:
#创建下载路径的文件夹
os.makedirs(download_path, exist_ok=True)

# 开始下载
error_log = download_spectra(url_txt_path, download_path, progress_file)

Number of spectra to download: 664821
-1 / unknown|| 5074 err: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
-1 / unknown|| 6048 err: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
-1 / unknown|| 7249 err: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
-1 / unknown|| 9357 err: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
-1 / unknown|| 9700 err: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
-1 / unknown|| 27776 err: HTTP Error 502: Bad Gateway
|| 27777 err: HTTP Error 502: Bad Gateway
|| 27778 err: HTTP Error 502: Bad Gateway
|| 27779 err: HTTP Error 502: Bad Gateway
|| 27780 err: HTTP Error 502: Bad Gateway
|| 27781 err: HTTP Error 502: Bad Gateway
|| 27782 err: HTTP Error 502: Bad Gateway
|| 27783 err: HTTP Error 502: Bad Gateway
|| 27784 err: HTTP Error 502: Bad Gateway
|| 27785 err: HTTP Error 502: Bad Gateway
|| 27786 err: HTTP Error 502: Bad Gateway
|| 27787 err: HTTP